<a href="https://colab.research.google.com/github/Angelvj/Alzheimer-disease-classification/blob/main/code/experiments1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
kaggle = False

# Imports

In [ ]:
import shutil
import sys
import tensorflow as tf

if tf.io.gfile.exists('Alzheimer-disease-classification'):
    shutil.rmtree('Alzheimer-disease-classification')
! git clone https://github.com/Angelvj/Alzheimer-disease-classification.git

if not kaggle:
    sys.path.insert(0,'/content/Alzheimer-disease-classification/code')
else:
    sys.path.insert(0, './Alzheimer-disease-classification/code')

In [3]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold
if not kaggle:
    from google.colab import drive
else:
    from kaggle_datasets import KaggleDatasets
    from kaggle_secrets import UserSecretsClient

import functions.lr_schedules as lr_schedules
import functions.io_utils as io
import functions.data_augmentation as augmentation
from functions.tfrec_loading import get_dataset
from functions.model_evaluation import repeated_kfold, plot_epochs_history, get_rkf_history, get_predictions, calculate_results, train_model, present_results
from functions.general import change_input_shape
from models import feedforward_models_pet, feedforward_models_mri

# Hardware config.

In [4]:
DEVICE = 'TPU' # or GPU
tpu = None

if DEVICE == 'TPU':
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        STRATEGY = tf.distribute.experimental.TPUStrategy(tpu)
    except ValueError:
        print('Could not connect to TPU, setting default strategy')
        tpu = None
        STRATEGY = tf.distribute.get_strategy()
elif DEVICE == 'GPU':
    STRATEGY = tf.distribute.MirroredStrategy()
    
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = STRATEGY.num_replicas_in_sync

print(f'Number of accelerators: {REPLICAS}')

Could not connect to TPU, setting default strategy
Number of accelerators: 1


# Constants

In [ ]:
SEED = 268 # Arbitrary seed

# Name of different datasets used
TFREC_DATASETS = ['tfrec-pet-spatialnorm-elastic-maxintensitynorm',
                  'tfrec-mri_grey-standarized-downscale',
                  'tfrec-mri_grey-standarized',
                  'tfrec-pet-minmax',
                  'tfrec-pet-standarized',
                  'tfrec-pet-minmax-standarized',
                  'tfrec-pet-maxintensitynorm',
                  'tfrec-pet'
                 ]

# Shape of images on each dataset
SHAPES = [(79, 95, 68, 1), (75, 90, 75, 1), (121, 145, 121, 1), (79, 95, 68, 1),
          (79, 95, 68, 1), (79, 95, 68, 1), (79, 95, 68, 1), (79, 95, 68, 1)]

# Number of repetitions and folds for repeated k-fold
REPS = 5
FOLDS = 10

# Different classes on the dataset
CLASSES = ['NOR', 'AD', 'MCI']

# Path to data
if kaggle:
    INPUT_DATAPATH = '/kaggle/input/' if tpu is None else None
    METADATA_PATH = '/kaggle/input/'
else:
    drive.mount('/content/drive') 
    INPUT_DATAPATH = '/content/drive/MyDrive/data/'
    METADATA_PATH = '/content/drive/MyDrive/data/'


# Function for setting up constant for a given dataset
def select_dataset(ds_id):
    global DS, IMG_SHAPE, DS_PATH, INPUT_DATAPATH, X_train, y_train, X_test, y_test
    DS = TFREC_DATASETS[ds_id]
    IMG_SHAPE = SHAPES[ds_id]
    if INPUT_DATAPATH == None:
        user_secrets = UserSecretsClient()
        user_credential = user_secrets.get_gcloud_credential()
        user_secrets.set_tensorflow_credential(user_credential)
        DS_PATH = KaggleDatasets().get_gcs_path(DS)
    else:
        DS_PATH = INPUT_DATAPATH + DS

    # Load filenames and labels
    metadata_train = pd.read_csv(METADATA_PATH + DS + '/train/train_summary.csv', encoding='utf-8')
    metadata_test = pd.read_csv(METADATA_PATH + DS + '/test/test_summary.csv', encoding='utf-8')
    X_train = DS_PATH + '/train/' + metadata_train.iloc[:, 0].to_numpy()
    y_train = np.argmax(metadata_train.iloc[:,-len(CLASSES):].to_numpy(), axis=1)
    X_test = DS_PATH + '/test/' + metadata_test.iloc[:, 0].to_numpy()
    y_test = np.argmax(metadata_test.iloc[:,-len(CLASSES):].to_numpy(), axis=1)

#  Phase 1: depth study

## PET

In [ ]:
select_dataset(0) # Preprocessed PET dataset

### Experiment 1 --> 1 convolutional layer

In [ ]:
BATCH_SIZE = 8
EPOCHS = 50
LR = 0.00001
DECAY = 0.1

detailed_history = repeated_kfold(feedforward_models_pet.model_0, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
# save_dict(detailed_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-0_results.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-0.txt')

### Experiment 2 --> 2 concolutional layers

In [ ]:
BATCH_SIZE = 4
EPOCHS = 50
LR = 0.00001
DECAY = 0.1

detailed_history = repeated_kfold(feedforward_models_pet.model_1, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
# save_dict(detailed_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-1_results.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-1.txt')

### Experiment 3 --> 3 convolutional layers

In [ ]:
BATCH_SIZE = 8
EPOCHS = 50
LR = 0.0001
DECAY = 0.01

detailed_history = repeated_kfold(feedforward_models_pet.model_2, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
# save_dict(detailed_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-2_results.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-2.txt')

### Experiment 4 --> 6 convolutional layers

In [ ]:
BATCH_SIZE = 4
EPOCHS = 40
LR = 0.000001
DECAY = 0.1

detailed_history = repeated_kfold(feedforward_models_pet.model_3, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
# save_dict(detailed_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-3_results.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-3.txt')

### Experiment 5 --> 8 convolutional layers

In [ ]:
BATCH_SIZE = 4
EPOCHS = 50
LR = 0.00001
DECAY = 0.1

detailed_history = repeated_kfold(feedforward_models_pet.model_4, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
# save_dict(detailed_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-4_results.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-4.txt')

# Train and get results on test data
model_4 = train_model(feedforward_models_pet.model_4, X_train, IMG_SHAPE, STRATEGY,
                      AUTO, BATCH_SIZE, EPOCHS, cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
y_pred = get_predictions(model_4, X_test, IMG_SHAPE, 3, AUTO, BATCH_SIZE)
results = calculate_results(y_test, y_pred)
present_results(results, {0:'NOR', 1:'AD', 2:'MCI'})
io.save_dict(results, 'model_4-pet-results.txt')

## MRI

In [ ]:
select_dataset(1) # Preprocessed MRI dataset, downscaled

### Experiment 1 --> 1 convolutional layer

In [ ]:
BATCH_SIZE = 4
EPOCHS = 50
LR = 0.000001
DECAY = 0.1

detailed_history = repeated_kfold(feedforward_models_mri.model_0, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
# save_dict(detailed_history, 'mri-grey-standarized-downscale_model-0.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'mri-grey-standarized-downscale_model-0.txt')

### Experiment 2 --> 3 convolutional layers

In [ ]:
BATCH_SIZE = 4
EPOCHS = 70
LR = 0.00001
DECAY = 0.1

detailed_history = repeated_kfold(feedforward_models_mri.model_1, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
# save_dict(detailed_history, 'mri-grey-standarized-downscale_model-1.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'mri-grey-standarized-downscale_model-1.txt')

### Experiment 3 --> 4 convolutional layers

In [ ]:
BATCH_SIZE = 4
EPOCHS = 50
LR = 0.000001
DECAY = 0.1

detailed_history = repeated_kfold(feedforward_models_mri.model_2, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
# save_dict(detailed_history, 'mri-grey-standarized-downscale_model-2.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'mri-grey-standarized-downscale_model-2.txt')

### Experiment 4 --> 6 convolutional layers

In [ ]:
BATCH_SIZE = 4
EPOCHS = 50
LR = 0.000001
DECAY = 0.1

detailed_history = repeated_kfold(feedforward_models_mri.model_3, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
# save_dict(detailed_history, 'mri-grey-standarized-downscale_model-3.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'mri-grey-standarized-downscale_model-3.txt')

### Experiment 5 --> 9 convolutional layers

In [ ]:
select_dataset(2) # Preprocessed MRI dataset, not downscaled

BATCH_SIZE = 4
EPOCHS = 50
LR = 0.000001
DECAY = 0.1

detailed_history = repeated_kfold(feedforward_models_mri.model_4, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
# save_dict(detailed_history, 'mri-grey-standarized-downscale_model-4.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'mri-grey-standarized_model-4.txt')

# Train and get results on test data
model_4 = train_model(feedforward_models_mri.model_4, X_train, IMG_SHAPE, STRATEGY,
                      AUTO, BATCH_SIZE, EPOCHS, cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
y_pred = get_predictions(model_4, X_test, IMG_SHAPE, 3, AUTO, BATCH_SIZE)
results = calculate_results(y_test, y_pred)
present_results(results, {0:'NOR', 1:'AD', 2:'MCI'})
io.save_dict(results, 'model_4-mri-results.txt')

# Phase 2: Data augmentation

## PET

In [ ]:
# Define augmentation for pet images
def augment_image(img):
    img = img.squeeze()
    original_shape = img.shape
    img = augmentation.random_rotations(img, -0.5, 0.5)
    img = augmentation.random_shift(img, 0.2)
    img = augmentation.downscale(img, original_shape)
    img = np.expand_dims(img, axis=3) 
    return img

# This function allow us to use numpy (augmentation) functions in training time with tensorflow
@tf.function(input_signature=[tf.TensorSpec(None, tf.float32)])
def tf_augment_image(input):
    img = tf.numpy_function(augment_image, [input], tf.float32)
    return img

select_dataset(0) # Preprocessed PET dataset

BATCH_SIZE = 8
EPOCHS = 50
LR = 0.0001
DECAY = 0.1 

detailed_history = repeated_kfold(feedforward_models_pet.model_4_augmentation, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)],
                         augment=tf_augment_image)
# save_dict(detailed_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-4_results.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'pet-spatialnorm-elastic-maxintensitynorm_model-4_augmentation.txt')

# Train and get results on test data
model_4 = train_model(feedforward_models_pet.model_4_augmentation, X_train, IMG_SHAPE, STRATEGY,
                      AUTO, BATCH_SIZE, EPOCHS, cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)],
                      augment=tf_augment_image)
y_pred = get_predictions(model_4, X_test, IMG_SHAPE, 3, AUTO, BATCH_SIZE)
results = calculate_results(y_test, y_pred)
present_results(results, {0:'NOR', 1:'AD', 2:'MCI'})
io.save_dict(results, 'model-4_augmentation_pet_results.txt')

## MRI

In [ ]:
# Define augmentation for pet images
def augment_image(img):
    img = img.squeeze()
    original_shape = img.shape
    img = augmentation.random_rotations(img, -0.5, 0.5)
    img = augmentation.random_zoom(img, 0.95, 1.05)
    img = augmentation.random_shift(img, 0.2)
    img = augmentation.downscale(img, original_shape)
    img = np.expand_dims(img, axis=3) 
    return img

# This function allow us to use numpy (augmentation) functions in training time with tensorflow
@tf.function(input_signature=[tf.TensorSpec(None, tf.float32)])
def tf_augment_image(input):
    img = tf.numpy_function(augment_image, [input], tf.float32)
    return img

select_dataset(2) # Preprocessed MRI dataset, not downscaled

BATCH_SIZE = 4
EPOCHS = 50
LR = 0.000001
DECAY = 0.1

# Data augmentation hyperparams
MAX_ROTATION = 0.5
ZOOM = (0.95, 1.05)
SHIFT = 0.2

detailed_history = repeated_kfold(feedforward_models_mri.model_4, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)],
                         augment=tf_augment_image)
# save_dict(detailed_history, 'mri-grey-standarized_model-4_augmentation.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'mri-grey-standarized_model-4_augmentation.txt')

# Train and get results on test data
model_4 = train_model(feedforward_models_mri.model_4, X_train, IMG_SHAPE, STRATEGY,
                      AUTO, BATCH_SIZE, EPOCHS, cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)],
                      augment=tf_augment_image)
y_pred = get_predictions(model_4, X_test, IMG_SHAPE, 3, AUTO, BATCH_SIZE)
results = calculate_results(y_test, y_pred)
present_results(results, {0:'NOR', 1:'AD', 2:'MCI'})
io.save_dict(results, 'model-4_augmentation_mri_results.txt')

# Phase 3: Raw images (only PET)

In [ ]:
BATCH_SIZE = 4
EPOCHS = 50
LR = 0.00001
DECAY = 0.1

select_dataset(0) # Really bad results

detailed_history = repeated_kfold(feedforward_models_pet.model_4, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)

select_dataset(4) # Better but equally bad

detailed_history = repeated_kfold(feedforward_models_pet.model_4, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)

select_dataset(5) # Again, bad results

detailed_history = repeated_kfold(feedforward_models_pet.model_4, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)

select_dataset(6) # Bad

detailed_history = repeated_kfold(feedforward_models_pet.model_4, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)

select_dataset(7) # The worst, as expected

detailed_history = repeated_kfold(feedforward_models_pet.model_4, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)])
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)

# Phase 4: Transfer learning

Note: for running this experiment, a pretrained resnet should be located at /content/drive/MyDrive/pretrained_models/pretrained_3D_resnet18.h5 (run the notebook pretrain_resnet.ipynb)

In [17]:
def build_pretrained_resnet18(input_shape):
    # Load pretrained resnet 
    pretrained_resnet18 = tf.keras.models.load_model('/content/drive/MyDrive/pretrained_models/pretrained_3D_resnet18.h5')
    # Delete classifier
    pretrained_resnet18 = tf.keras.Model(pretrained_resnet18.input, pretrained_resnet18.layers[-2].output)
    # Change input shape to match our dataset
    pretrained_resnet18 = change_input_shape(pretrained_resnet18, IMG_SHAPE, name='new_input')

    pretrained_resnet18.trainable = False
    inputs = tf.keras.layers.Input(shape=IMG_SHAPE, name='pet_input')
    features = pretrained_resnet18(inputs, training=False)
    # Add new classifier at the end
    outputs = tf.keras.layers.Dense(3)(features)
    pretrained_resnet18 = tf.keras.Model(inputs, outputs)
    return pretrained_resnet18

## PET

In [ ]:
select_dataset(0) # Preprocessed PET dataset

# Configure augmentation
def augment_image(img):
    img = img.squeeze()
    original_shape = img.shape
    img = augmentation.random_rotations(img, -0.5, 0.5)
    img = augmentation.random_shift(img, 0.2)
    img = augmentation.downscale(img, original_shape)
    img = np.expand_dims(img, axis=3) 
    return img

@tf.function(input_signature=[tf.TensorSpec(None, tf.float32)])
def tf_augment_image(input):
    img = tf.numpy_function(augment_image, [input], tf.float32)
    return img

BATCH_SIZE = 4
EPOCHS = 30
LR = 0.0001
DECAY = 1 # Not decay

detailed_history = repeated_kfold(build_pretrained_resnet18, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)],
                         augment=tf_augment_image)
# save_dict(detailed_history, 'pet-spatialnorm-elastic-maxintensitynorm_resnet18_results.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
io.save_dict(rkf_history, 'pet-spatialnorm-elastic-maxintensitynorm_resnet18.txt')

## MRI

In [ ]:
select_dataset(2) # Preprocessed MRI dataset, not downscaled

# Configure augmentation
def augment_image(img):
    img = img.squeeze()
    original_shape = img.shape
    img = augmentation.random_rotations(img, -0.5, 0.5)
    img = augmentation.random_zoom(img, 0.95, 1.05)
    img = augmentation.random_shift(img, 0.2)
    img = augmentation.downscale(img, original_shape)
    img = np.expand_dims(img, axis=3) 
    return img

@tf.function(input_signature=[tf.TensorSpec(None, tf.float32)])
def tf_augment_image(input):
    img = tf.numpy_function(augment_image, [input], tf.float32)
    return img

BATCH_SIZE = 4
EPOCHS = 30
LR = 0.0001
DECAY = 1 # Not decay

detailed_history = repeated_kfold(build_pretrained_resnet18, X_train, y_train, 
                         IMG_SHAPE, STRATEGY, tpu, AUTO, FOLDS, BATCH_SIZE, EPOCHS, REPS, 
                         cbks=[lr_schedules.get_exp_lr_decay_callback(LR, DECAY, EPOCHS)],
                         augment=tf_augment_image)
# save_dict(detailed_history, 'mri-grey-standarized_resnet18_results.txt')
rkf_history = get_rkf_history(detailed_history)
plot_epochs_history(EPOCHS, rkf_history)
mri-grey-standarized_resnet_augmentation.txt
io.save_dict(rkf_history, 'mri-grey-standarized_resnet18.txt')